# Hello CSI Camera

## CSI (Camera Serial Interface) Cameras

This notebook is designed to test that the camera is functioning as expected. Of course, the camera must be connected to the MIPI CSI port, make sure the flex is in good condition and remove any obstructions on the camera lens such as a film or cover.

## Check to see if the device is available

Execute the following system command to list all video devices on the Jetson Nano. If your camera doesn't show up with a device id, check your connection. You should get an output similar to

```
crw-rw----+ 1 root video 81, 0 Jan 11 14:08 /dev/video0
```

In [ ]:
!ls -ltrh /dev/video*

## Create the camera object

First, create a camera object by importing the `CSICamera` class from the library by executing the following Python code cell. Please note, you can only create one `CSICamera` instance.

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, flip_method=6)

In [ ]:
image = camera.read()

print(image.shape)

## Create a widget to view the image stream

The next step is to create a widget to render the image coming from the camera.

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')

image_widget.value = bgr8_to_jpeg(image)

display(image_widget)

You should see an image from the camera if all is working correctly. If there seems to be an image but it's fuzzy or a funny color, check to make sure there is no protective film or cap on the lens.

Now let's watch a live stream from the camera. Set the `running` value of the camera to continuously update the value in background. This allows you to attach "callbacks" to the camera value changes.

The "callback" here is the function, `update_image`, which is attached by calling the `observe` method below. `update_image` is executed whenever there is a new image available to process, which is then displayed in the widget.

In [ ]:
camera.running = True

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    
camera.observe(update_image, names='value')

If you move something in front of the camera, you should now see the live video stream in the widget. To stop it, unattach the callback with the unobserve method.

In [ ]:
camera.unobserve(update_image, names='value')

> **Tip** You can move the widgets (or any cell) to new window tabs in JupyterLab by right-clicking the cell and selecting "Create New View for Output". This way, you can continue to scroll down the JupyterLab notebook and still see the camera view

## Another way to view the image stream

In [ ]:
import traitlets

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

You can remove the camera/widget link with the unlink method.

In [ ]:
camera_link.unlink()

## Shut down the kernel of this notebook to release the camera resource. !!